# VAE Latent Risk Factor - Pipeline Dashboard

Central configuration and execution notebook for the full walk-forward validation pipeline.

**Workflow:**
1. Configure all parameters (Sections 1-2)
2. Load data (Section 3)
3. Run pipeline (Section 4)
4. Inspect results (Sections 5-7)

---
## 1. Setup

In [ ]:
import os
import sys
import logging
from dataclasses import replace, asdict
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

# Project root: go up from notebooks/ to project root
_NB_DIR = Path(os.path.abspath("")).resolve()
PROJECT_ROOT = (_NB_DIR / "..").resolve() if _NB_DIR.name == "notebooks" else _NB_DIR
os.chdir(PROJECT_ROOT)
sys.path.insert(0, str(PROJECT_ROOT))

from src.config import (
    PipelineConfig,
    DataPipelineConfig,
    VAEArchitectureConfig,
    LossConfig,
    TrainingConfig,
    InferenceConfig,
    RiskModelConfig,
    PortfolioConfig,
    WalkForwardConfig,
)
from src.data_pipeline.data_loader import load_data_source
from src.data_pipeline.returns import compute_log_returns
from src.data_pipeline.features import compute_trailing_volatility
from src.integration.pipeline import FullPipeline
from src.integration.reporting import export_results, format_summary_table
from src.integration.visualization import (
    plot_fold_metrics,
    plot_e_star_distribution,
    plot_pairwise_heatmap,
    style_summary_table,
    style_fold_table,
)
from src.utils import get_optimal_device
from src.walk_forward.selection import aggregate_fold_metrics, summary_statistics

%matplotlib inline
plt.rcParams["figure.dpi"] = 120

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(name)s: %(message)s")
logger = logging.getLogger("dashboard")

print(f"PyTorch {torch.__version__} | Device: {get_optimal_device()}")
print(f"Working directory: {os.getcwd()}")

---
## 2. Configuration

Two configuration profiles are available. **Run ONLY one section:**
- **Section 2a** — Synthetic data: minimal parameters for quick end-to-end testing
- **Section 2b** — Real data: full production configuration

Always run the **Global** cell (below) first, then choose ONE section.

In [ ]:
# ============================================================
# GLOBAL
# ============================================================
SEED = 42
DEVICE = str(get_optimal_device())

# Data source: "synthetic", "tiingo", or "csv"
DATA_SOURCE = "tiingo"
QUICK_MODE = True  # Set True for minimal config even with real data

# Tiingo API keys (used when DATA_SOURCE = "tiingo")
TIINGO_API_KEYS = [
    "9ba6e57788deaac3b3c38ed47047cabbbd6077e2",
    "9aad315d49275c400687f41dd26b22328d8b1a26",
]
DATA_DIR = "data/"  # Directory for Tiingo downloaded data

### 2a. Quick Mode

Run **only this cell** to configure the pipeline for a minimal end-to-end test. Skip Section 2b entirely.

In [ ]:
# ============================================================
# SYNTHETIC — Minimal config for end-to-end testing
# Run ONLY this cell, then jump to Section 3
# ============================================================

if QUICK_MODE == True or DATA_SOURCE == "synthetic":
    DATA_PATH = ""
    N_STOCKS = 50
    N_YEARS = 20

    # K=10 for speed; r_max relaxed because C_L floor (384) makes the CNN
    # too large for small synthetic universes — acceptable for testing only.
    config = PipelineConfig(
        data=DataPipelineConfig(
            n_stocks=N_STOCKS,
            window_length=504,
            n_features=2,
        ),
        vae=VAEArchitectureConfig(
            K=100,
            window_length=504,
            n_features=2,
            r_max=1e6,
        ),
        loss=LossConfig(mode="P"),
        training=TrainingConfig(
            max_epochs=50,
            batch_size=256,
            learning_rate=1e-4,
            patience=30,
        ),
        inference=InferenceConfig(),
        risk_model=RiskModelConfig(),
        portfolio=PortfolioConfig(n_starts=2),
        walk_forward=WalkForwardConfig(
            total_years=N_YEARS,
            min_training_years=max(3, N_YEARS // 3), # At least 3 years to avoid too-short training sets in early folds
            holdout_years=max(1, N_YEARS // 5), # At least 1 year holdout to ensure we have a test set in each fold
        ),
        seed=SEED,
    )

    HP_GRID = [{"mode": "P", "learning_rate": 1e-4, "alpha": 1.0}]

    print(f"[Quick mode] {N_STOCKS} stocks, {N_YEARS} years, K={config.vae.K}")
    print(f"  max_epochs={config.training.max_epochs}, patience={config.training.patience}, HP_GRID=1 config, n_starts=2")
    print(f"  r_max={config.vae.r_max:.0e} (relaxed for testing)")
    print(f"  Walk-forward: {config.walk_forward.total_years}y total, "
        f"{config.walk_forward.min_training_years}y min training, "
        f"{config.walk_forward.holdout_years}y holdout")
    print(f"  Device: {DEVICE}")

### 2b. Real Data (Production)

Run **all cells below** (through "ASSEMBLE FULL CONFIG") for full production configuration. Skip Section 2a.

In [ ]:
# ============================================================
# DATA SOURCE — Real data
# ============================================================
# For CSV source:
DATA_PATH = "data/stock_data.csv"  # <-- Set path to your stock data CSV

# For Tiingo source: run download first:
#   python scripts/download_tiingo.py --phase all --keys-file keys.txt
# Then set DATA_SOURCE = "tiingo" in Global cell above.

# Universe and history parameters (overridden by synthetic cell if DATA_SOURCE == "synthetic")
N_STOCKS = 50                    # Top N stocks by median market cap (50=fast, 200=realistic, 0=all)
N_YEARS = 14                     # Years of history to keep (0=all)

In [ ]:
# ============================================================
# DATA PIPELINE (MOD-001)
# ============================================================
data_cfg = DataPipelineConfig(
    n_stocks=N_STOCKS,           # universe cap (same as N_STOCKS above)
    window_length=504,           # T: sliding window length (trading days)
    n_features=2,                # F: features per timestep (return + realized vol)
    vol_window=252,              # trailing vol lookback (days)
    vix_lookback_percentile=80.0,# VIX percentile for crisis threshold
    min_valid_fraction=0.80,     # minimum valid data fraction per stock
)

In [ ]:
# ============================================================
# VAE ARCHITECTURE (MOD-002)
# ============================================================
vae_cfg = VAEArchitectureConfig(
    K=200,                       # latent capacity ceiling
    sigma_sq_init=1.0,           # initial observation noise
    sigma_sq_min=1e-4,           # lower clamp for sigma^2
    sigma_sq_max=10.0,           # upper clamp for sigma^2
    window_length=504,           # T (must match data_cfg)
    n_features=2,                # F (must match data_cfg)
    r_max=5.0,                   # max parameter/data ratio (relaxed for synthetic)
)

print(f"Encoder depth L={vae_cfg.encoder_depth}, "
      f"Final width C_L={vae_cfg.final_layer_width}, "
      f"D={vae_cfg.D}")

In [ ]:
# ============================================================
# LOSS FUNCTION (MOD-004)
# ============================================================
loss_cfg = LossConfig(
    mode="P",                    # 'P' (primary), 'F' (fallback), 'A' (advanced)
    gamma=3.0,                   # crisis overweighting factor
    lambda_co_max=0.5,           # max co-movement loss weight
    beta_fixed=1.0,              # fixed beta for Mode A
    warmup_fraction=0.20,        # fraction of epochs for Mode F warmup
    max_pairs=2048,              # max pairs for co-movement loss
    delta_sync=21,               # max date gap for synchronization (days)
)

In [ ]:
# ============================================================
# TRAINING (MOD-005)
# ============================================================
training_cfg = TrainingConfig(
    max_epochs=100,              # maximum training epochs
    batch_size=256,              # batch size
    learning_rate=1e-4,          # initial learning rate (eta_0)
    weight_decay=1e-5,           # Adam weight decay
    adam_betas=(0.9, 0.999),     # Adam betas
    adam_eps=1e-8,               # Adam epsilon
    patience=10,                 # early stopping patience (epochs)
    lr_patience=5,               # ReduceLROnPlateau patience
    lr_factor=0.5,               # ReduceLROnPlateau factor
    n_strata=15,                 # strata for synchronous batching
    curriculum_phase1_frac=0.30, # fraction of epochs: sync+stratified batching
    curriculum_phase2_frac=0.30, # fraction of epochs: + co-movement loss ramp
)

In [ ]:
# ============================================================
# INFERENCE (MOD-006)
# ============================================================
inference_cfg = InferenceConfig(
    batch_size=512,              # inference batch size
    au_threshold=0.01,           # KL threshold for active unit (nats)
    r_min=2,                     # min observations-per-parameter for AU_max
    aggregation_method="mean",   # profile aggregation method
)

In [ ]:
# ============================================================
# RISK MODEL (MOD-007)
# ============================================================
risk_model_cfg = RiskModelConfig(
    winsorize_lo=5.0,            # lower percentile for vol ratio winsorization
    winsorize_hi=95.0,           # upper percentile
    d_eps_floor=1e-6,            # floor for idiosyncratic variance
    conditioning_threshold=1e6,  # condition number threshold for ridge fallback
    ridge_scale=1e-6,            # ridge regularization scale
)

In [ ]:
# ============================================================
# PORTFOLIO OPTIMIZATION (MOD-008)
# Constraints identical for VAE and all 6 benchmarks (INV-012)
# ============================================================
portfolio_cfg = PortfolioConfig(
    lambda_risk=1.0,             # risk aversion
    w_max=0.05,                  # max weight per stock (hard cap)
    w_min=0.001,                 # min active weight (semi-continuous)
    w_bar=0.03,                  # concentration penalty threshold
    phi=25.0,                    # concentration penalty weight
    kappa_1=0.1,                 # linear turnover penalty
    kappa_2=7.5,                 # quadratic turnover penalty
    delta_bar=0.01,              # turnover penalty threshold
    tau_max=0.30,                # max one-way turnover (hard cap)
    n_starts=5,                  # multi-start initializations
    sca_max_iter=100,            # max SCA iterations
    sca_tol=1e-8,               # SCA convergence tolerance
    armijo_c=1e-4,               # Armijo sufficient decrease
    armijo_rho=0.5,              # Armijo backtracking factor
    armijo_max_iter=20,          # max Armijo backtracking steps
    max_cardinality_elim=100,    # max cardinality elimination rounds
    entropy_eps=1e-30,           # numerical stability for log()
    alpha_grid=[0.0, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0],  # frontier alpha grid
)

In [ ]:
# ============================================================
# WALK-FORWARD VALIDATION (MOD-009)
# ============================================================
walk_forward_cfg = WalkForwardConfig(
    total_years=N_YEARS,              # total history length
    min_training_years=10,       # minimum training window
    oos_months=6,                # out-of-sample period (months)
    embargo_days=21,             # embargo between train and OOS (trading days)
    holdout_years=3,             # final holdout period
    val_years=2,                 # nested validation for Phase A
    score_lambda_pen=5.0,        # MDD penalty weight in composite score
    score_lambda_est=2.0,        # estimation quality penalty weight
    score_mdd_threshold=0.20,    # MDD threshold in composite score
)

In [ ]:
# ============================================================
# HP GRID for Phase A (set to None for default 18-config grid)
# ============================================================
HP_GRID = None  # None = default: 3 modes x 2 LRs x 3 alphas = 18 configs

# Uncomment to define a custom grid:
# HP_GRID = [
#     {"mode": "P", "learning_rate": 5e-4, "alpha": 1.0},
#     {"mode": "F", "learning_rate": 1e-3, "alpha": 0.5},
#     {"mode": "A", "learning_rate": 1e-3, "alpha": 2.0},
# ]

In [ ]:
# ============================================================
# ASSEMBLE FULL CONFIG
# ============================================================
if DATA_SOURCE != "synthetic" and QUICK_MODE == False:
      config = PipelineConfig(
            data=data_cfg,
            vae=vae_cfg,
            loss=loss_cfg,
            training=training_cfg,
            inference=inference_cfg,
            risk_model=risk_model_cfg,
            portfolio=portfolio_cfg,
            walk_forward=walk_forward_cfg,
            seed=SEED,
      )

      print("PipelineConfig assembled.")
      print(f"  Walk-forward: {config.walk_forward.total_years}y total, "
            f"{config.walk_forward.min_training_years}y min training, "
            f"{config.walk_forward.holdout_years}y holdout")
      print(f"  VAE: K={config.vae.K}, T={config.vae.window_length}, F={config.vae.n_features}")
      print(f"  Training: {config.training.max_epochs} max epochs, "
            f"bs={config.training.batch_size}, lr={config.training.learning_rate}")
      print(f"  Loss mode: {config.loss.mode}, gamma={config.loss.gamma}")
      print(f"  Capacity guard r_max: {config.vae.r_max}")
      print(f"  Device: {DEVICE}")

---
## 3. Data Loading

### 3a. Tiingo Download (run once)

Run this cell **only once** to download Tiingo data. After the first run, the data is saved locally and reused automatically.
Set `MAX_TICKERS` to a small number (e.g. 5) for testing, or `None` for the full universe.

In [ ]:
# ============================================================
# TIINGO DOWNLOAD — Run once, data is saved locally
# Skip this cell if data is already downloaded or DATA_SOURCE != "tiingo"
# ============================================================

if DATA_SOURCE == "tiingo":
    import importlib.util

    _spec = importlib.util.spec_from_file_location(
        "download_tiingo", str(PROJECT_ROOT / "scripts" / "download_tiingo.py")
    )
    _mod = importlib.util.module_from_spec(_spec)
    _spec.loader.exec_module(_mod)  # type: ignore[union-attr]

    MAX_TICKERS = None  # Set to None for full universe (~22k tickers)

    _mod.run_download(
        api_keys=TIINGO_API_KEYS,
        data_dir=DATA_DIR,
        max_tickers=MAX_TICKERS,
        sp500_first=True,  # Download SP500 tickers first (priority)
    )
else:
    print(f"DATA_SOURCE={DATA_SOURCE}, skipping Tiingo download.")

In [ ]:
np.random.seed(SEED)

stock_data, start_date = load_data_source(
    source=DATA_SOURCE,
    data_path=DATA_PATH if DATA_SOURCE == "csv" else "",
    data_dir=DATA_DIR,
    n_stocks=N_STOCKS,
    n_years=N_YEARS,
    seed=SEED,
)

print(f"Data source: {DATA_SOURCE}")
print(f"Stock data shape: {stock_data.shape}")
print(f"Date range: {stock_data['date'].min()} to {stock_data['date'].max()}")
print(f"Unique stocks: {stock_data['permno'].nunique()}")
stock_data.head()

In [ ]:
# Compute log-returns and trailing volatility
returns = compute_log_returns(stock_data)
trailing_vol = compute_trailing_volatility(returns, window=config.data.vol_window)

print(f"Returns: {returns.shape[0]} dates x {returns.shape[1]} stocks")
print(f"Trailing vol: {trailing_vol.shape} (first {config.data.vol_window-1} rows NaN)")
print(f"Returns date range: {returns.index[0]} to {returns.index[-1]}")

---
## 4. Run Pipeline

Executes the full walk-forward validation: Phase A (HP selection) + Phase B (deployment) on each fold, then benchmarks, statistical tests, and report generation.

### TensorBoard Monitoring

Launch TensorBoard in a terminal to monitor training in real-time:

```bash
.venv/bin/tensorboard --logdir runs/
```

Logs are organized as `runs/fold_XX/phase_a/config_YY_mode_M_lr_L/` and `runs/fold_XX/phase_b/`.

**Step-level metrics** (logged every 50 batches) — raw per-batch values, useful to spot instabilities:

| Metric | What to look for |
|--------|-----------------|
| `Step/loss` | Should decrease overall. Spikes are normal early on. |
| `Step/reconstruction` | Main driver of total loss. Should decrease steadily. |
| `Step/kl_divergence` | Grows as the VAE learns to use latent dimensions. Too high = posterior collapse risk. |
| `Step/co_movement` | Ramps up during curriculum phase 2 (epochs 30-60%). Should stabilize. |
| `Step/sigma_sq` | Learned observation noise. Should settle between 0.01-1.0. |

**Epoch-level metrics** (logged once per epoch) — smoothed averages, useful for overall trends:

| Metric | What to look for |
|--------|-----------------|
| `Loss/total` | Overall training loss (lower = better). |
| `Validation/ELBO` | Key metric for early stopping. Gap with training loss = overfitting. |
| `Training/active_units` | Number of latent dimensions actually used (KL > 0.01 nats). Target: 15-80% of K. |
| `Training/learning_rate` | Should step down when validation plateaus (ReduceLROnPlateau). |
| `Training/lambda_co` | Co-movement weight schedule: 0 during phase 1, ramps to lambda_co_max during phase 2. |
| `Training/beta_t` | Beta annealing (Mode F only): 0 to 1 during warmup, then fixed at 1. |

**Quick diagnostic:**
- **Good training**: reconstruction drops, AU grows to 20-60, validation tracks training
- **Posterior collapse**: AU stays near 0, KL near 0 — try Mode F or lower learning rate
- **Overfitting**: validation diverges from training — early stopping should trigger

In [ ]:
# Print training configuration summary
print("=" * 60)
print("PIPELINE CONFIGURATION SUMMARY")
print("=" * 60)
print(f"  Seed: {config.seed} | Device: {DEVICE} | Data: {DATA_SOURCE}")
print()
print(f"  [Data]         n_stocks={config.data.n_stocks}, T={config.data.window_length}, "
      f"F={config.data.n_features}, vol_window={config.data.vol_window}")
print(f"  [VAE]          K={config.vae.K}, L={config.vae.encoder_depth}, "
      f"C_L={config.vae.final_layer_width}, r_max={config.vae.r_max}")
print(f"  [Loss]         mode={config.loss.mode}, gamma={config.loss.gamma}, "
      f"lambda_co_max={config.loss.lambda_co_max}, beta_fixed={config.loss.beta_fixed}")
print(f"  [Training]     max_epochs={config.training.max_epochs}, bs={config.training.batch_size}, "
      f"lr={config.training.learning_rate}, wd={config.training.weight_decay}")
print(f"                 patience={config.training.patience}, lr_patience={config.training.lr_patience}, "
      f"lr_factor={config.training.lr_factor}")
print(f"                 curriculum: phase1={config.training.curriculum_phase1_frac}, "
      f"phase2={config.training.curriculum_phase2_frac}, strata={config.training.n_strata}")
print(f"  [Inference]    bs={config.inference.batch_size}, AU_threshold={config.inference.au_threshold}, "
      f"r_min={config.inference.r_min}")
print(f"  [Risk Model]   winsorize=[{config.risk_model.winsorize_lo}, {config.risk_model.winsorize_hi}], "
      f"cond_threshold={config.risk_model.conditioning_threshold:.0e}")
print(f"  [Portfolio]    w_max={config.portfolio.w_max}, w_min={config.portfolio.w_min}, "
      f"tau_max={config.portfolio.tau_max}, n_starts={config.portfolio.n_starts}")
print(f"  [Walk-Forward] {config.walk_forward.total_years}y total, "
      f"{config.walk_forward.min_training_years}y min train, "
      f"{config.walk_forward.oos_months}mo OOS, "
      f"{config.walk_forward.holdout_years}y holdout, "
      f"embargo={config.walk_forward.embargo_days}d")
n_hp = len(HP_GRID) if HP_GRID else "18 (default)"
print(f"  [HP Grid]      {n_hp} configs")
print("=" * 60)

TB_DIR = "runs/"  # TensorBoard log directory (set to None to disable)

pipeline = FullPipeline(config, tensorboard_dir=TB_DIR)

results = pipeline.run(
    stock_data=stock_data,
    returns=returns,
    trailing_vol=trailing_vol,
    skip_phase_a=(DATA_SOURCE == "synthetic" or QUICK_MODE == True),
    vix_data=None,
    start_date=start_date,
    hp_grid=HP_GRID,
    device=DEVICE,
)

print("Pipeline complete.")
print(f"Folds processed: {len(results['vae_results'])}")
print(f"Benchmarks: {list(results['benchmark_results'].keys())}")

---
## 5. Results - Summary Report

In [ ]:
# Text summary
print(format_summary_table(results["report"]))

In [ ]:
# Deployment recommendation
deployment = results["report"]["deployment"]
print(f"Scenario: {deployment['scenario']}")
print(f"Recommendation: {deployment['recommendation']}")
print()
print("Per-benchmark wins (VAE vs benchmark on primary metrics):")
for bench, info in deployment["per_benchmark"].items():
    print(f"  {bench:20s}: {info['wins']}/{info['total']} metrics won")

In [ ]:
# VAE summary statistics
vae_df = aggregate_fold_metrics(results["vae_results"])
vae_summary = summary_statistics(vae_df)
print("VAE Summary Statistics:")
style_summary_table(vae_summary)

In [ ]:
# Benchmark summary statistics
for bench_name, bench_metrics in results["benchmark_results"].items():
    bench_df = aggregate_fold_metrics(bench_metrics)
    bench_summary = summary_statistics(bench_df)
    print(f"\n{bench_name} Summary:")
    display(style_summary_table(bench_summary))

---
## 6. Results - Per-Fold Detail

In [ ]:
# VAE per-fold metrics
print("VAE Per-Fold Metrics:")
style_fold_table(vae_df)

In [ ]:
# E* distribution
e_star_summary = results["report"]["e_star_summary"]
print(f"E* epochs: mean={e_star_summary['mean']:.1f}, "
      f"std={e_star_summary['std']:.1f}, "
      f"range=[{e_star_summary['min']}, {e_star_summary['max']}]")

plot_e_star_distribution(results["e_stars"])
plt.show()

In [ ]:
# Fold metrics: VAE vs benchmarks
plot_fold_metrics(results["vae_results"], results["benchmark_results"])
plt.show()

---
## 7. Results - Statistical Tests

In [ ]:
# Pairwise tests heatmap
plot_pairwise_heatmap(results["report"])
plt.show()

In [ ]:
# Detailed pairwise test results
tests = results["report"]["statistical_tests"]
print(f"Total comparisons: {tests['n_tests']} (alpha={tests['alpha']})")
print()

for bench_name, metrics in tests["pairwise"].items():
    print(f"VAE vs {bench_name}:")
    for metric, result in metrics.items():
        if result.get("skipped", False):
            print(f"  {metric}: skipped ({result['reason']})")
            continue
        sig = " *" if result.get("significant_corrected", False) else ""
        print(f"  {metric}: delta={result['median_delta']:+.4f} "
              f"[{result['ci_lower']:+.4f}, {result['ci_upper']:+.4f}] "
              f"p={result.get('p_corrected', result['p_value']):.4f}{sig}")
    print()

---
## 8. Export Results

In [ ]:
OUTPUT_DIR = "results/"

written = export_results(results, asdict(config), output_dir=OUTPUT_DIR)

print(f"Results saved to {OUTPUT_DIR}")
for path in written:
    print(f"  {os.path.basename(path)}")